In [ ]:
import get_doctors_from_db
from doctor_class import doctor
import numpy as np
import openpyxl
import os
import sqlite3
import pandas as pd
import datetime


создается класс "доктор" с параметрами id врача, доступные модальности, ставка, график работы (5\2, 1\1, 2\2), его смены на месяц и количество часов, которые он в эти смены работает.

In [ ]:
class doctor:
    def __init__(self,id,abilities, working_rate,graphic_of_work, available_days, amount_of_hours, is_used ):
        self.id = id
        self.abilities = abilities
        self.working_rate = working_rate
        self.graphic_of_work = graphic_of_work
        self.available_days = []
        for d in available_days:
            self.available_days.append(d[:])
        self.is_used = is_used
        self.amount_of_hours = []          #количество рабочих часов в неделю, тоэже вроде безполезный параметр

        self.time_schedule = []             #параметр для вычисления продуктивности, тип 8 часов - 1, 10 часов в смене - 1.2
        self.time_out = []
        i = 0
        for week in self.available_days:
            self.amount_of_hours.append([])
            self.time_schedule.append([])
            for day in week:

                self.amount_of_hours[i].append(amount_of_hours[i]* day  * self.working_rate )
                self.time_schedule[i].append(amount_of_hours[i] * day * self.working_rate / 8)
            i+=1
        #print(self.time_schedule)
        i = 0
        for week in self.amount_of_hours:
            self.time_out.append([])
            for pereriv in week:
                if pereriv > 8:
                    self.time_out[i].append(1)
                else:
                    self.time_out[i].append(0.5)
            i += 1

        self.activities_per_day = []              #количество оставшейся энергии врача
        self.new_available_days = []              #новое расписание на каждый день в месяце
        self.work_per_day = []                    #какие ивенты врач выполнял за день

        i = 0

        for week in available_days:
            self.activities_per_day.append([])
            self.new_available_days.append([])
            self.work_per_day.append([])
            j = 0
            for day in week:
                if day > 0:
                    self.activities_per_day[i].append( 300 *self.time_schedule[i][j])
                else:
                    self.activities_per_day[i].append(0)
                self.new_available_days[i].append(0)
                self.work_per_day[i].append([])
                j+=1
            i += 1

    def set_new_available_days(self, new_schedule, week, day):
        self.new_available_days[week][day] = new_schedule

    def get_new_available_days(self):

        return self.new_available_days

    def get_work_per_day(self):
        return self.work_per_day

    def get_activities_per_day(self,week,day):
        return self.activities_per_day[week][day]

    def set_work_per_day(self, new_schedule, week, day):
        self.work_per_day[week][day].append(new_schedule)

    def set_activities_per_day(self, new_schedule, week, day):
        self.activities_per_day[week][day] = new_schedule

    def get_pereriv(self,week):
        return self.time_out[week]

    def set_pereriv(self, shedule, week):
        self.time_out[week] = shedule

    def get_pereriv_per_day(self,week,day):
        return self.time_out[week][day]

    def set_pereriv_per_day(self, shedule, week, day):
        self.time_out[week][day] = shedule

    def get_id(self):
        return self.id

    def get_abilities(self):
        return self.abilities

    def get_working_rate(self):
        # print(self.working_rate)
        return self.working_rate

    def get_available_days(self):
        return self.available_days

    def get_graphic_of_work(self):
        return self.graphic_of_work

    def get_amount_of_hours(self,week):
        return self.amount_of_hours[week]
    def get_amount_of_hours_per_day(self,week,day):
        return self.amount_of_hours[week][day]

    def get_used(self):
        return self.is_used

    def get_available_week_schedule(self, week):
        return self.available_days[week]

    def get_available_day_schedule(self, week, day):
        return self.available_days[week][day]

    def get_week_schedule(self, week):
        return self.time_schedule[week]

    def get_day_schedule(self, week, day):
        return self.time_schedule[week][day]

    def get_schedule(self):
        return self.time_schedule

    def set_new_day_schedule(self, week, day, new_schedule):
        self.available_days[week][day] = new_schedule

    def set_new_week_schedule(self, week, new_schedule):
        self.available_days[week] = new_schedule

    def set_available_days(self, new_schedule):
        self.available_days = new_schedule

    def set_used(self, flag):
        self.is_used = flag

    def set_new_abilities(self,abilities):
        self.abilities = abilities

    def set_new_working_rate(self,rate):
        self.working_rate = rate



    def vivod(self):
        print(str(self.id) + "  " + str(self.abilities) + "  " + str(self.working_rate) + "  " + str(self.graphic_of_work) + "  " + str(self.available_days) + "  " + str(self.amount_of_hours))


файл "get_doctors_from_db.py" содержит в себе функции для считывания данных о докторах из базы данных и создания массива переменных класса "доктор", в котором содержатся все доктора

In [ ]:
def create_doctors(all_doctors):
    all_docs = []

    for doc in all_doctors:
        #print(doc)
        all_docs.append(doctor(doc[0], doc[1], doc[2], doc[3], doc[4], doc[5],True))

    return all_docs

def coun():
    # Подключение к базе данных
    conn = sqlite3.connect('doc_list.db')  # Укажите название вашей базы данных
    # Создание курсора
    cursor = conn.cursor()
    # Запрос на подсчет числа строк в таблице
    table_name = 'users'  # Укажите название вашей таблицы
    cursor.execute(f"SELECT COUNT(*) FROM doc_list")
    # Получение результата
    row_count = cursor.fetchone()[0]
    # Закрытие соединения
    conn.close()
    return row_count

def read_limited_rows():
    try:
        all_mas = []
        sqlite_connection = sqlite3.connect('db.sqlite3')
        cursor = sqlite_connection.cursor()
        cursor.execute(f"SELECT COUNT(*) FROM accounts_employee")
        # Получение результата
        row_count = cursor.fetchone()[0]
        sqlite_select_query = """SELECT * from accounts_employee"""
        cursor.execute(sqlite_select_query)
        records = cursor.fetchmany(row_count)
        j=0
        for i in range(0, len(records)):
            all_mas.append([])
            #print(records[i])
            for a in range(0,len(records[i])):
                if (records[i][1] != 1) | (records[i][2] == '') :
                    all_mas.pop(i-j)
                    j+=1
                    break
                else:
                    if a == 0:
                        all_mas[i-j].append(records[i][a])
                    if a == 2:
                        if records[i-j][a] == 'КТ':
                            all_mas[i-j].append(['КТ'])
                            all_mas[i-j][1].append('КТ1')
                            all_mas[i-j][1].append('КТ2')
                        elif records[i][a] == 'МРТ':
                            all_mas[i-j].append(['МРТ'])
                            all_mas[i-j][1].append('МРТ1')
                            all_mas[i-j][1].append('МРТ2')
                        else:
                            all_mas[i-j].append([str(records[i][a])])
                    if (a == 3) & (records[i][a] != None) & (records[i][a] != '-'):
                        tx = records[i][a].split(',')
                        for mes in tx:
                            if mes.strip() == 'Денс':
                                all_mas[i-j][1].append('Денситометрия')
                            elif mes.strip() == 'Денситометрия':
                                all_mas[i-j][1].append('Денситометрия')
                            elif mes.strip() == 'РГ':
                                all_mas[i-j][1].append('РГ')
                            elif mes.strip() == 'КТ':
                                all_mas[i-j][1].append('КТ')
                                all_mas[i-j][1].append('КТ1')
                                all_mas[i-j][1].append('КТ2')
                            elif mes.strip() == 'ММГ':
                                all_mas[i-j][1].append('ММГ')
                            elif mes.strip() == 'ФЛГ':
                                all_mas[i-j][1].append('ФЛГ')
                            elif mes.strip() == 'МРТ':
                                all_mas[i-j][1].append('МРТ')
                                all_mas[i-j][1].append('МРТ1')
                                all_mas[i-j][1].append('МРТ2')
                    if a == 4:
                        all_mas[i-j].append(records[i][a])
                    if a == 5:
                        all_mas[i-j].append([int(m) for m in records[i][a][1:len(records[i][a])-1].split(', ')])
                    if a == 6:
                        A = records[i][a][2:-2]
                        # Разделяем строку по '], ['
                        list_str = A.split('], [')
                        # Преобразуем каждую строку внутри вложенного списка в список целых чисел
                        array_2d = [[int(num) for num in sublist.split(', ')] for sublist in list_str]
                        all_mas[i-j].append(array_2d)

                    if a == 7:
                        all_mas[i-j].append([int(m) for m in records[i][a][1:len(records[i][a])-1].split(', ')])

        cursor.close()

    except sqlite3.Error as error:
        print("Ошибка при работе с SQLite", error)
    finally:
        if sqlite_connection:
            sqlite_connection.close()

    return all_mas

print(read_limited_rows())


def get_values():

    doctors = read_limited_rows()
    all_docs = create_doctors(doctors)


    return all_docs

в файле create_schedule.py создается расписание врача. Суть алгоритма:
Рассматривается каждый день месяца, для каждого дня считается усредненное значение, которое нужно выполнить по каждой из модальностей. находится модальность с минимальном количеством врачей, после чего по очереди врачи заполняются на ее выполнение. как только модальность кончается, переходит к следующей минимальной, пока все не закончится.


In [ ]:
def check_for_overload(activities, schetchik, amount_for_month):
    mas = {}
    for key in schetchik.keys():
        mas[key] = amount_for_month[key] - schetchik[key] * activities[key] * 5
    return mas

def make_schetchik(all_docs, schetchik):
    new_schet = {}
    for key in schetchik.keys():
        new_schet[key] = 0

    for key in schetchik.keys():
        for doc in all_docs:
            if key in doc.get_abilities():
                new_schet[key] += 1
    return new_schet


def find_min_key(schetchik):
    min_key = min(schetchik, key=schetchik.get)
    #print(min_key)
    return min(schetchik, key=schetchik.get)

def find_doctors(all_docs, key, week, day):
    mas_of_suitable_doctors = []
    i = 0
    for doc in all_docs:
        #print(doc.get_available_days())
        #print(str(week) + '   '+ str(day) + '   ' + str(doc.get_id()))
        if doc.get_activities_per_day(week, day)>0:
            if key in doc.get_abilities() :
                mas_of_suitable_doctors.append(i)
        i+=1
    return mas_of_suitable_doctors

def find_priority_doctors(all_docs, key, week, day):
    mas_of_doctors_in_priority = [[], []]
    doctors = find_doctors(all_docs,key, week, day)
    for i in doctors:
        if len(all_docs[i].get_abilities()) == 1:
            mas_of_doctors_in_priority[0].append(i)
        else:
            mas_of_doctors_in_priority[1].append(i)

    return mas_of_doctors_in_priority

def update_schetchik(schetchik, all_docs, week,day):
    schetchik_new = {}
    for key in schetchik.keys():
        schetchik_new[key] = 0
    for doc in all_docs:
        if doc.get_activities_per_day(week, day) > 0:
            for key in doc.get_abilities():
                if key in schetchik_new.keys():
                    schetchik_new[key] += 1

    return schetchik_new

def zapolnenie_exel(all_docs,amount_for_weeks):
    workbook = openpyxl.load_workbook('doc.xlsx')
    worksheet = workbook.active
    book = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL']
    doc_id = 0
    schetchik_for_docs = 0
    days_in_month = 0
    for week in range(0, len(amount_for_weeks)):
        for day in range(0, len(all_docs[0].get_available_week_schedule(week))):
            days_in_month += 1

    for doc in all_docs:
        date = 3
        shetchik_time = 0
        worksheet['A' + str(4*schetchik_for_docs +3)] = doc.get_id()
        schetchik_for_docs +=1

        for week in range(0, len(amount_for_weeks)):

            for day in range(0, len(all_docs[0].get_available_week_schedule(week))):
                strin1 = str(book[date-3 +6]) + str(4 * doc_id + 3 )
                strin2 = str(book[date-3 +6]) + str(4 * doc_id + 4)
                strin3 = str(book[date-3 +6]) + str(4 * doc_id + 5)
                strin4 = str(book[date-3 +6]) + str(4 * doc_id + 6)



                if doc.get_amount_of_hours_per_day(week, day) == 12:
                    worksheet[strin1] = str('20:00')
                    worksheet[strin2] = str('9:00')
                    worksheet[strin3] = str((all_docs[doc_id].get_pereriv(week)[day] % 1 + int(all_docs[doc_id].get_pereriv(week)[day])) * 60)

                    timee = all_docs[doc_id].get_amount_of_hours_per_day(week, day) + all_docs[doc_id].get_pereriv(week)[day]

                    shetchik_time+=12
                    worksheet[strin4] = 12

                elif doc.get_amount_of_hours_per_day(week, day) == 0:
                    worksheet[strin1].value = 0
                    worksheet[strin2].value = 0
                    worksheet[strin3].value = 0
                    worksheet[strin4].value = 0
                else:
                    timee = all_docs[doc_id].get_amount_of_hours_per_day(week, day) + all_docs[doc_id].get_pereriv(week)[day]
                    worksheet[strin1] = str('8:00')
                    worksheet[strin2] = str(8 + int(timee)) + ':' + str(int((timee % 1) * 60))
                    worksheet[strin3] = str((all_docs[doc_id].get_pereriv(week)[day] % 1 + int(all_docs[doc_id].get_pereriv(week)[day])) * 60)



                    worksheet[strin4] = all_docs[doc_id].get_amount_of_hours_per_day(week, day)
                    shetchik_time += all_docs[doc_id].get_amount_of_hours_per_day(week, day)

                print(date)

                if date-3 == 15:
                    strin = 'V' + str(4 * doc_id + 6)
                    worksheet[strin] = shetchik_time
                    shetchik_time = 0
                elif date-3 == days_in_month-1:
                    strin = 'AM' + str(4 * doc_id + 6)
                    worksheet[strin] = shetchik_time
                    shetchik_time = 0

                date = date + 1


        doc_id += 1

    workbook.save('file_to_download.xlsx')

def kostil(amount_for_week, worksheet, days_in_month, kost, book, schetchik):
    worksheet['A1'] = ''
    worksheet['B1']= ''
    worksheet['C1']= ''
    worksheet['D1']= ''
    worksheet['E1']= ''
    worksheet['F1']= ''
    worksheet['G1']= ''
    worksheet['H1']= ''
    worksheet['I1']= ''
    worksheet['J1']= ''

    strin1 = str(book[0]) + str(kost+1)
    strin2 = str(book[1]) + str(kost+1)
    strin3 = str(book[2]) + str(kost+1)
    strin4 = str(book[3]) + str(kost+1)
    strin5 = str(book[4]) + str(kost+1)
    strin6 = str(book[5]) + str(kost+1)
    strin7 = str(book[6]) + str(kost+1)
    strin8 = str(book[7]) + str(kost+1)
    strin9 = str(book[8]) + str(kost+1)
    strin10 = str(book[9]) + str(kost+1)
    keys = [key for key in schetchik.keys()]
    if int(amount_for_week[keys[0]]) > 0:
        worksheet[strin1] = str(keys[0])+":"+str(int(amount_for_week[keys[0]]))
    else:
        worksheet[strin1] = str(keys[0])+":"+str(int(0))
    if int(amount_for_week[keys[1]]) > 0:
        worksheet[strin2] = str(keys[1])+":"+str(int(amount_for_week[keys[1]]))
    else:
        worksheet[strin2] = str(keys[1])+":"+str(int(0))
    if int(amount_for_week[keys[2]]) > 0:
        worksheet[strin3] = str(keys[2])+":"+str(int(amount_for_week[keys[2]]))
    else:
        worksheet[strin3] = str(keys[2])+":"+str(int(0))
    if int(amount_for_week[keys[3]]) > 0:
        worksheet[strin4] = str(keys[3])+":"+str(int(amount_for_week[keys[3]]))
    else:
        worksheet[strin4] = str(keys[3])+":"+str(int(0))
    if int(amount_for_week[keys[4]]) > 0:
        worksheet[strin5] = str(keys[4])+":"+str(int(amount_for_week[keys[4]]))
    else:
        worksheet[strin5] = str(keys[4])+":"+str(int(0))
    if int(amount_for_week[keys[5]]) > 0:
        worksheet[strin6] = str(keys[5])+":"+str(int(amount_for_week[keys[5]]))
    else:
        worksheet[strin6] = str(keys[5])+":"+str(int(0))
    if int(amount_for_week[keys[6]]) > 0:
        worksheet[strin7] = str(keys[6])+":"+str(int(amount_for_week[keys[6]]))
    else:
        worksheet[strin7] = str(keys[6])+":"+str(int(0))
    if int(amount_for_week[keys[7]]) > 0:
        worksheet[strin8] = str(keys[7])+":"+str(int(amount_for_week[keys[7]]))
    else:
        worksheet[strin8] = str(keys[7])+":"+str(int(0))
    if int(amount_for_week[keys[8]]) > 0:
        worksheet[strin9] = str(keys[8])+":"+str(int(amount_for_week[keys[8]]))
    else:
        worksheet[strin9] = str(keys[8])+":"+str(int(0))
    if int(amount_for_week[keys[9]]) > 0:
        worksheet[strin10] = str(keys[9])+":"+str(int(amount_for_week[keys[9]]))
    else:
        worksheet[strin10] = str(keys[9])+":"+str(int(0))


def make_schedule_for_month(all_docs, activities, activities_UE, amount_for_weeks,schetchik):
    max = 300
    workbook1 = openpyxl.Workbook()
    worksheet1 = workbook1.active
    workbook = openpyxl.Workbook()
    worksheet = workbook.active
    book = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
                'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL']
    days_in_month = 0
    for week in range(0, len(amount_for_weeks)):
        for day in range(0, len(all_docs[0].get_available_week_schedule(week))):
            days_in_month+=1
    kost = 1
    for week in range(0, len(amount_for_weeks)):

        for day in range(0, len(all_docs[0].get_available_week_schedule(week))):
            amount_for_week = dict(amount_for_weeks[week])
            print('start:')
            print(amount_for_week)

            schetchik_for_day = update_schetchik(schetchik, all_docs, week,day)
            print(schetchik_for_day)
            flag = len(schetchik_for_day)

            while flag > 0:

                min_key = find_min_key(schetchik_for_day)

                massive_of_doctors_in_priority_order = find_priority_doctors(all_docs, min_key,week,day)

                for priority_order in massive_of_doctors_in_priority_order:
                    if amount_for_week[min_key] <= 0:
                        break
                    for doc_id in priority_order:

                        if (amount_for_week[min_key] <= 0) | (all_docs[doc_id].get_activities_per_day(week, day) <= 0):
                            break
                        while all_docs[doc_id].get_activities_per_day(week, day) > 0:
                            if amount_for_week[min_key] <= 0:
                                break
                            amount_for_week[min_key] = amount_for_week[min_key] - (all_docs[doc_id].get_day_schedule(week, day) * activities_UE[min_key])

                            #all_docs[doc_id].set_new_day_schedule(week, day, 0)                                                             #UBIRAET DEN
                            new_amount = all_docs[doc_id].get_activities_per_day(week, day) - (all_docs[doc_id].get_day_schedule(week, day) * activities_UE[min_key])
                            #print(str(min_key) + '  '+ str(new_amount) +  '  ' + str(amount_for_week[min_key]))
                            all_docs[doc_id].set_activities_per_day(new_amount, week, day)                                                   #SKOLKO ENERGYY VRACHA OSTALOS

                        all_docs[doc_id].set_new_available_days(all_docs[doc_id].get_available_day_schedule(week, day),week, day)      #RASPISANIE DLA ZAPOLNENIA TABLIZ
                        all_docs[doc_id].set_work_per_day(min_key, week, day)                                                           #KAKOE ISSLEDOVANIE DELAET

                schetchik_for_day.pop(min_key)
                schetchik_for_day = update_schetchik(schetchik_for_day, all_docs, week,day)
                flag-=1
            print('end:')
            print(amount_for_week)
            kostil(amount_for_week, worksheet, days_in_month, kost, book, schetchik)
            kost += 1
            print(schetchik_for_day)
    doc_id = 0
    schetchik_for_docs = 0
    workbook.save('kostil.xlsx')
    df = pd.read_excel('kostil.xlsx')

    conn = sqlite3.connect('db.sqlite3')
    # Запись данных из Excel файла в базу данных
    df.to_sql('overload', conn, if_exists='replace', index=True)
    # Завершение работы с базой данных
    conn.commit()
    conn.close()

    worksheet1['A1'] = 'sys_user'
    worksheet1['B1'] = 'day_of_month'
    worksheet1['C1'] = 'time_start'
    worksheet1['D1'] = 'time_end'
    worksheet1['E1'] = 'time_break'
    worksheet1['F1'] = 'time_total'
    worksheet1['G1'] = 'research_type'
    for doc in all_docs:
        date = 1
        for week in range(0, len(amount_for_weeks)):

            for day in range(0, len(all_docs[0].get_available_week_schedule(week))):
                strin1 = str(book[0]) + str(days_in_month * doc_id + 1 + date)
                strin2 = str(book[1]) + str(days_in_month * doc_id + 1 + date)
                strin3 = str(book[2]) + str(days_in_month * doc_id + 1 + date)
                strin4 = str(book[3]) + str(days_in_month * doc_id + 1 + date)
                strin5 = str(book[4]) + str(days_in_month * doc_id + 1 + date)
                strin6 = str(book[5]) + str(days_in_month * doc_id + 1 + date)
                strin7 = str(book[6]) + str(days_in_month * doc_id + 1 + date)

                worksheet1[strin1] = str(all_docs[doc_id].get_id())
                worksheet1[strin2] = str(date)



                if doc.get_amount_of_hours_per_day(week,day) == 12:
                    worksheet1[strin3] = str('20:00')
                    worksheet1[strin4] = str('9:00')
                    worksheet1[strin5] = str((all_docs[doc_id].get_pereriv(week)[day] % 1 + int(
                        all_docs[doc_id].get_pereriv(week)[day])) * 60)
                    timee = all_docs[doc_id].get_amount_of_hours_per_day(week,day) + all_docs[doc_id].get_pereriv(week)[day]

                    worksheet1[strin6] = str(int(timee)) + ':' + str(int((timee % 1) * 60))
                    worksheet1[strin7] = str(doc.get_work_per_day()[week][day])

                elif doc.get_amount_of_hours_per_day(week,day) == 0:
                    worksheet1[strin3] = str(0)
                    worksheet1[strin4] = str(0)
                else:
                    worksheet1[strin3] = str('8:00')
                    timee = all_docs[doc_id].get_amount_of_hours_per_day(week,day) + all_docs[doc_id].get_pereriv(week)[day]
                    worksheet1[strin4] = str(8 + int(timee)) + ':' + str(int((timee % 1) * 60))
                    worksheet1[strin5] = str((all_docs[doc_id].get_pereriv(week)[day] % 1 + int(
                        all_docs[doc_id].get_pereriv(week)[day])) * 60)
                    worksheet1[strin6] = str(int(timee)) + ':' + str(int((timee % 1) * 60))
                    worksheet1[strin7] = str(doc.get_work_per_day()[week][day])
                #print(date)
                date = date + 1

        doc_id+=1

    workbook1.save('test2.xlsx')
    df = pd.read_excel('test2.xlsx', header=0)

    conn = sqlite3.connect('db.sqlite3')
    # Запись данных из Excel файла в базу данных
    df.to_sql('time_schedule', conn, if_exists='replace', index=True)
    # Завершение работы с базой данных
    conn.commit()
    conn.close()

    zapolnenie_exel(all_docs, amount_for_weeks)

        #print(week)
    #os.remove("test2.xlsx")
    return 0

def make_schetchik_for_all(all_docs):
    new_schetchik = {}
    for doc in all_docs:
        for key in doc.get_abilities():
            if key in new_schetchik.keys():
                new_schetchik[key] += 1
            else:
                new_schetchik[key] = 1
    return new_schetchik

def dop_smena_ravnomerno(all_docs, amount_for_week, activities):
    schetchik = make_schetchik_for_all(all_docs)
    amount_of_hours_per_doc = {}
    for key, value in amount_for_week.items():
        if value > 0:
            amount_of_hours_per_doc[key] = (8 * value) / (activities[key] * schetchik[key])

    return amount_of_hours_per_doc



'''ORDER BY rowid DESC LIMIT 4'''





def get_month_of_week(year, week):
    d = datetime.date(year, 1, 1)
    d = d + datetime.timedelta(weeks=week - 1, days=-d.weekday())
    return d.month


def get_stuff(year, week):
    get_month_of_week(year, week)
    month = get_month_of_week(year, week)

    first_day = datetime.date(year, month, 1)
    first_week_number = first_day.isocalendar()[1]
    return first_week_number


def get_amount_for_weeks(schetchik, activities_UE):
    conn = sqlite3.connect('db.sqlite3')  # Укажите название вашей базы данных
    cursor = conn.cursor()
    my_table = 'accounts_excelmodel'
    year = cursor.execute('SELECT * FROM accounts_excelmodel ORDER BY "index" DESC LIMIT 1 OFFSET 1;')
    year = year.fetchone()[1]
    week = cursor.execute('SELECT * FROM accounts_excelmodel ORDER BY "index" DESC LIMIT 1 OFFSET 1;')
    week = week.fetchone()[2]
    week_to_start_from = get_stuff(year,week)
    print(week_to_start_from)
    cursor.execute(f'SELECT * FROM accounts_excelmodel ORDER BY "index" DESC LIMIT {week_to_start_from}')
    last_four_rows = cursor.fetchall()

    # Заполнение массива данными
    data_array = []
    for i in range(0, len(last_four_rows)):
        data_array.append(last_four_rows[-1-i])

    amount_of_weeks = []
    a = 0
    for row in data_array:
        i = 3

        amount_of_weeks.append({})
        for key in schetchik.keys():
            if row[i] % 7 < 3.5:
                amount_of_weeks[a][key] = int(row[i] / 7 * activities_UE[key])
            else:
                amount_of_weeks[a][key] = int((int(row[i] / 7) + 1) * activities_UE[key])

            i+=1
        a+=1

    return amount_of_weeks


#181   300   451

def main():
    all_docs = get_doctors_from_db.get_values()
    schetchik = {'Денситометрия': 0, 'КТ': 0, 'КТ1': 0, 'КТ2': 0, 'ММГ': 0, 'МРТ': 0, 'МРТ1': 0, 'МРТ2': 0, 'РГ': 0, 'ФЛГ': 0}
    activities_max_150 = {'Денситометрия': 210, 'КТ': 39, 'КТ1': 24, 'КТ2': 17, 'ММГ': 123, 'МРТ': 30, 'МРТ1': 23,
                          'МРТ2': 15, 'РГ': 123, 'ФЛГ': 451}  # Словарь со всеми исследованиями и их значениями

    activities = {'Денситометрия': 140, 'КТ': 26, 'КТ1': 16, 'КТ2': 11, 'ММГ': 82, 'МРТ': 20, 'МРТ1': 15, 'МРТ2': 10,
                  'РГ': 82, 'ФЛГ': 300}  # Словарь со всеми исследованиями и их значениями
    activities_UE = {'Денситометрия': 2.4, 'КТ': 11.6, 'КТ1': 18.8, 'КТ2': 26.6, 'ММГ': 3.7, 'МРТ': 15.1, 'МРТ1': 19.7, 'МРТ2': 30.1,
                  'РГ': 3.7, 'ФЛГ': 1}  # Словарь со всеми исследованиями и их значениями in UA

    for key in activities_max_150:
        activities[key] = int(activities_max_150[key] / 1.5)
    print(activities)
    schetchik = make_schetchik(all_docs,schetchik)
    print(schetchik)

    amount_for_week = get_amount_for_weeks(schetchik, activities_UE)       #2022 15 данные для тестов
    print(amount_for_week)
    print(make_schedule_for_month(all_docs, activities,activities_UE, amount_for_week, schetchik))


main()
